In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# -*- coding: utf-8 -*-
"""

"""

import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
import os 
import cv2
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


use_GPU = 0
if use_GPU:
 	physical_devices = tf.config.list_physical_devices('GPU')
 	tf.config.experimental.set_memory_growth(physical_devices[0], True)
     
NUMBER_OF_LABELS = 10
IMG_WIDTH = 224
IMG_HEIGHT = 224
CHANNELS = 3
LOAD_MODEL = 1
NUMBER_PATTERN_FOR_TRAINING = 250
PERCENTAGE_OF_TRAINING_IMAGES=0.01

training_path = "/content/drive/MyDrive/job_recognition/DataSet/train"
MODEL_PATH = '/content/drive/MyDrive/Colab Notebooks/job_recog_modelmodel/model1.h5'
#============================#
#Load data#
#============================#

#====================#
#LOAD OR CREATE MODEL#
#====================#
def train(X_train, y_train, X_valid, y_valid):
    if LOAD_MODEL:
        '''Loading model'''
        model = keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/job_recog_model")
    else:
        #create model
        model = keras.models.Sequential([
            keras.layers.Input(shape=((IMG_WIDTH, IMG_HEIGHT, CHANNELS))),
            keras.layers.Conv2D(32, kernel_size=3, padding='same', activation='relu'),
            keras.layers.Conv2D(64, kernel_size=3, padding='same', activation='relu'),
            keras.layers.BatchNormalization(),
            keras.layers.MaxPooling2D(2),
            
            keras.layers.Conv2D(128, kernel_size=3, padding='same', activation='relu'),
            keras.layers.Conv2D(128, kernel_size=3, padding='same', activation='relu'),
            keras.layers.BatchNormalization(),
            keras.layers.MaxPooling2D(2),
            
            keras.layers.Conv2D(256, kernel_size=3, padding='same', activation='relu'),
            keras.layers.Conv2D(256, kernel_size=3, padding='same', activation='relu'),
            keras.layers.BatchNormalization(),
            keras.layers.MaxPooling2D(2),
            
            keras.layers.Flatten(),
            keras.layers.Dense(128, activation='relu'),
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dense(NUMBER_OF_LABELS, activation='softmax')])
        
    model.summary()
    
    #============#
    #TRAIN MODEL#
    #============#
    
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=keras.optimizers.Adam(lr=0.01),
                  metrics=['accuracy'])
    
    #performace scheduling 
    lr_scheduling = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.25, patience=5)
    
    save_the_best_model = keras.callbacks.ModelCheckpoint(MODEL_PATH, monitor='val_loss', save_best_only=True)
    
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

    #train
    history = model.fit(np.array(X_train), np.array(y_train), epochs=30, 
                        validation_data=(np.array(X_valid), np.array(y_valid)), 
                        callbacks=[lr_scheduling, save_the_best_model, early_stopping])
    history = history.history


'''Loading data'''
label_names =['chef', 'doctor', 'engineer', 'farmer', 'firefighter',
              'judge', 'mechanic', 'pilot', 'police', 'waiter']

iteration = 0

while iteration*PERCENTAGE_OF_TRAINING_IMAGES<1:
    print("\nIter: ", iteration)
    images=[]
    labels = []
    for folder_name in label_names:
        folder_path = os.path.join(training_path, folder_name)
           
        all_images = os.listdir(folder_path)
        for img_name in all_images[int(iteration*PERCENTAGE_OF_TRAINING_IMAGES*len(all_images))
                                   :int((iteration+1)*PERCENTAGE_OF_TRAINING_IMAGES*len(all_images))]:
            #get image
            image = cv2.imread(os.path.join(folder_path, img_name))
            # print(image)
            # show image 
            # plt.imshow(image)
            # plt.show() 
            
            #check imageempty  
            if image is None:
                continue
                
            image = np.array(image)
            image = image.reshape((IMG_WIDTH, IMG_HEIGHT, CHANNELS))
            image =image/255.0
            images.append(image)
            labels.append(label_names.index(folder_name))
            
    iteration=+1      
    
    X_train, X_valid, y_train, y_valid = train_test_split(images, labels, test_size=0.2)
    
    train(X_train, y_train, X_valid, y_valid)







In [2]:

# -*- coding: utf-8 -*-
"""

"""

import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
import os 
import cv2
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


use_GPU = 0
if use_GPU:
 	physical_devices = tf.config.list_physical_devices('GPU')
 	tf.config.experimental.set_memory_growth(physical_devices[0], True)
     
NUMBER_OF_LABELS = 10
IMG_WIDTH = 224
IMG_HEIGHT = 224
CHANNELS = 3
LOAD_MODEL = 1
NUMBER_PATTERN_FOR_TRAINING = 250
PERCENTAGE_OF_TRAINING_IMAGES=0.25

training_path = "/content/drive/MyDrive/job_recognition/DataSet/train"
MODEL_PATH = '/content/drive/MyDrive/Colab Notebooks/job_recog_modelmodel/model3.h5'

#============================#
#Load data#
#============================#

#====================#
#LOAD OR CREATE MODEL#
#====================#
def residual_block(input, filters, kernel_size, adaptive_layer=False):
  '''class to represent residual architecture of resnet
  adaptive_layer: insert adaptive layer when input size is different with output size
  return 3 blocks'''

  residual_input = input

  if adaptive_layer:
    residual_input = keras.layers.Conv2D(filters, kernel_size=1, padding='same')(residual_input)
    residual_input = keras.layers.BatchNormalization()(residual_input)

  # input = keras.layers.Conv2D(filters, kernel_size=kernel_size, 
  #                             padding = 'same')(input)
  # input = keras.layers.BatchNormalization()(input)
  # input = keras.layers.Activation('relu')(input)
  
  input = keras.layers.Conv2D(filters, kernel_size=kernel_size, 
                              padding = 'same')(input)
  input = keras.layers.BatchNormalization()(input)
  input = keras.layers.Activation('relu')(input)

  input = keras.layers.Conv2D(filters, kernel_size=kernel_size, 
                              padding = 'same')(input)
  input = keras.layers.BatchNormalization()(input)

  input = keras.layers.Add()([input, residual_input])

  input = keras.layers.Activation('relu')(input)
  return input


def model(load_model, filter_array):
    '''Load or create model'''
    if LOAD_MODEL:
        '''Loading model'''
        model = keras.models.load_model(MODEL_PATH)
    else:
        input = keras.layers.Input(shape=(224, 224, 3))

        output = keras.layers.Conv2D(64, kernel_size=3, padding='same')(input)
        output = keras.layers.BatchNormalization()(output)
        output = keras.layers.Activation('relu')(output)

        output = keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(output)

        for i in range(len(filter_array)):
          #don't use adaptive layer
          if filter_array[i]==filter_array[max(0, i-1)]:
            output = residual_block(output, filter_array[i], 3, False)
          else:
            output = residual_block(output, filter_array[i], 3, True)

        output = keras.layers.GlobalAvgPool2D()(output)
        output = keras.layers.Dense(10)(output)
        output = keras.layers.Activation("softmax")(output)

        model = keras.models.Model(inputs=[input], outputs = [output])

    model.summary()
    return model


def train(model, X_train, y_train, X_valid, y_valid):
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=keras.optimizers.Adam(lr=0.01),
                  metrics=['accuracy'])
    
    #performace scheduling 
    lr_scheduling = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
    
    save_the_best_model = keras.callbacks.ModelCheckpoint(MODEL_PATH, monitor='val_loss', save_best_only=True)
    
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

    #train
    history = model.fit(np.array(X_train), np.array(y_train), epochs=60, 
                        validation_data=(np.array(X_valid), np.array(y_valid)), 
                        callbacks=[lr_scheduling, save_the_best_model, early_stopping])
    history = history.history


'''Loading data'''
label_names =['chef', 'doctor', 'engineer', 'farmer', 'firefighter',
              'judge', 'mechanic', 'pilot', 'police', 'waiter']

iteration = 2

model = model(0, [64, 64, 128, 256, 512] )
keras.utils.plot_model(model, "my_fashion_mnist_model.png", show_shapes=True)

while iteration*PERCENTAGE_OF_TRAINING_IMAGES<1:
    print("\nIter: ", iteration)
    images=[]
    labels = []
    for folder_name in label_names:
        folder_path = os.path.join(training_path, folder_name)
           
        all_images = os.listdir(folder_path)
        for img_name in all_images[int(iteration*PERCENTAGE_OF_TRAINING_IMAGES*len(all_images))
                                   :int((iteration+1)*PERCENTAGE_OF_TRAINING_IMAGES*len(all_images))]:
            #get image
            image = cv2.imread(os.path.join(folder_path, img_name))
            # print(image)
            # show image 
            # plt.imshow(image)
            # plt.show() 
            
            #check imageempty  
            if image is None:
                continue
                
            image = np.array(image)
            image = image.reshape((IMG_WIDTH, IMG_HEIGHT, CHANNELS))
            image =image/255.0
            images.append(image)
            labels.append(label_names.index(folder_name))
            
    iteration=iteration+1      
    
    X_train, X_valid, y_train, y_valid = train_test_split(images, labels, test_size=0.2)
    print("y_train: ", y_train)
    print("y_valid: ", y_valid)
    
    train(model, X_train, y_train, X_valid, y_valid)



Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 224, 224, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 224, 224, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 224, 224, 64) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [1]:
'''TEST'''

# -*- coding: utf-8 -*-
"""
Created on Wed May  5 23:41:30 2021

@author: nguye
"""

import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
import os 
import cv2
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

NUMBER_OF_LABELS = 10
IMG_WIDTH = 224
IMG_HEIGHT = 224
CHANNELS = 3

test_path = "/content/drive/MyDrive/job_recognition/DataSet/test"
MODEL_PATH = '/content/drive/MyDrive/Colab Notebooks/job_recog_modelmodel/model3.h5'


label_names =['chef', 'doctor', 'engineer', 'farmer', 'firefighter',
              'judge', 'mechanic', 'pilot', 'police', 'waiter']

#load model
model = keras.models.load_model(MODEL_PATH)

#=======================#
#Evaluate entire test set
#=======================#
if 1:
    images=[]
    labels = []
    for folder_name in label_names:
        folder_path = os.path.join(test_path, folder_name)
           
        all_images = os.listdir(folder_path)
        for img_name in all_images:
            #get image
            image = cv2.imread(os.path.join(folder_path, img_name))
            
            #check imageempty  
            if image is None:
                continue
                
            image = np.array(image)

            image = image.reshape((IMG_WIDTH, IMG_HEIGHT, CHANNELS))
            image =image/255.0
            images.append(image)
            labels.append(label_names.index(folder_name))   

    model.evaluate(np.array(images), np.array(labels))

#=======================================#
#for testing image gotten on the internet
#=======================================#
if 0:
    folder_path = os.path.join(test_path)
       
    all_images = os.listdir(folder_path)
    for img_name in all_images:
        #get image
        original_image = cv2.imread(os.path.join(folder_path, img_name))
        
        #check imageempty  
        if original_image is None:
            continue
            
        image = np.array(original_image)
        
        image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
        image =image/255.0
      
        pred = model.predict(np.array([image]))
        name=label_names[np.argmax(pred,axis=1)[0]]
        
        plt.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
        plt.title(name)
        plt.show()



63/63 [==============================] - 83s 672ms/step - loss: 1.0422 - accuracy: 0.7090
